In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for Gordon


No vtk


In [2]:
exclude_nodes = [33]

In [3]:
classifier_setting = 2

# From Patch Features to Score Volumes

In [ ]:
# for stack in all_stacks:
for stack in ['MD635']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    #################################

    t = time.time()
    sys.stderr.write('running svm classifier ...')


    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'apply_classifiers_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

    #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'interpolate_scoremaps_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False
    # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

    run_distributed4(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', '/visualize_scoremaps_v3.py'),
                    'stack': stack,
                     'setting': classifier_setting,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack


    #################################

    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'construct_score_volume_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(structure=all_known_structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds

    #################################

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))
    downscale = 32

    t1 = time.time()

    for structure in all_known_structures:
        try:
            volume = DataManager.load_score_volume(stack=stack, structure=structure, downscale=downscale, setting=classifier_setting)

            t = time.time()

            gy_gx_gz = np.gradient(volume.astype(np.float32), 3, 3, 3) 
            # 3.3 second - re-computing is much faster than loading
            # .astype(np.float32) is important; 
            # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
            # 2s (float32) vs. 20s (float16)

            sys.stderr.write('Gradient %s: %f seconds\n' % (structure, time.time() - t))

            t = time.time()

            gx_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, setting=classifier_setting, suffix='gx')
            gy_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, setting=classifier_setting, suffix='gy')
            gz_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, setting=classifier_setting, suffix='gz')

            bp.pack_ndarray_file(gy_gx_gz[1], gx_fp)
            bp.pack_ndarray_file(gy_gx_gz[0], gy_fp)
            bp.pack_ndarray_file(gy_gx_gz[2], gz_fp)

            del gy_gx_gz

            sys.stderr.write('save %s: %f seconds\n' % (structure, time.time() - t))

        except:
            sys.stderr.write('Score volume for %s does not exist.\n' % structure)

    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1)) # 150s

# Global align

In [5]:
warp_setting = 1

In [5]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'global_registration_v3.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 1000 seconds ~ 20 mins

align all subjects to atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 924.642506 seconds


In [8]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_global.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
                kwargs_list=dict(stack=all_nissl_stacks),
#                  kwargs_list=dict(stack=['MD592']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 150 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 146.914101 seconds


In [9]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                 kwargs_list=dict(stack=['MD594']),
                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 607.789498 seconds


# Local align

In [10]:
warp_setting = 2

In [11]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1300 seconds

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 1868.386311 seconds


In [12]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_local.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                 kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 245.859235 seconds


In [14]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 190 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 696.656998 seconds


# Transform locally transformed volumes back to atlas space

In [16]:
warp_setting = 3

In [18]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 198.215866 seconds


# Measure Global Tx Confidence

In [4]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

Measure confidence ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


# Measure Local Tx Confidence

In [10]:
warp_setting = 2

In [12]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 s ~ 15 mins

Measure confidence ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-45.sdsc.edu is down
gcn-20-47.sdsc.edu is down
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-48.sdsc.edu']


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/opt/python/lib/python2.7/inspect.py", line 1043, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/opt/python/lib/python2.7/inspect.py", line 1003, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/opt/python/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasa

IndexError: string index out of range

# Snake

In [6]:
train_sample_scheme

8

In [7]:
# Graphcut

t = time.time()
sys.stderr.write('Graphcut ...')

exclude_nodes = [33]

# for stack in all_stacks:
for stack in ['MD635']:

    run_distributed4(command='%(script_path)s %(stack)s %%(name_sided)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_contour_to_scoremap.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(name_sided=[convert_to_left_name(s) for s in paired_structures] + \
                                     [convert_to_right_name(s) for s in paired_structures] + \
                                     [s for s in singular_structures]),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Graphcut ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-31.sdsc.edu is down
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 163.411004 seconds


# Generate score volumes